In [1]:
# set up non-QM hardware
import Labber
# connect to server 
client = Labber.connectToServer('localhost') # get list of instruments 
#instruments = client.getListOfInstrumentsString() 
#for instr in instruments: 
#    print(instr) # close connection 

# reset all QDevil channels to 0 V
QDevil = client.connectToInstrument('QDevil QDAC', dict(interface='Serial', address='3'))
for n in range(24):
    if n+1 < 10:
        QDevil.setValue("CH0" + str(n+1) + " Voltage", 0.0)
    else:
        QDevil.setValue("CH" + str(n+1) + " Voltage", 0.0)
# setting CH06 to sweet spot
DC_ss = 6.4352E-3
QDevil.setValue("CH06 Voltage", DC_ss)

# digital attenuators
Vaunix1 = client.connectToInstrument('Painter Vaunix Lab Brick Digital Attenuator', dict(interface='USB', address='25606'))
Vaunix2 = client.connectToInstrument('Painter Vaunix Lab Brick Digital Attenuator', dict(interface='USB', address='25607'))
ROI = 42
ROO = 10
Vaunix1.setValue("Attenuation", ROI)
Vaunix2.setValue("Attenuation", ROO)

# TWPA pump
SG = client.connectToInstrument('Rohde&Schwarz RF Source', dict(interface='TCPIP', address='192.168.88.2'))
SG.setValue('Frequency', 6730.6E6)
SG.setValue('Power', -11.3)

client.close()

In [8]:
from qm.qua import *
from qm.QuantumMachinesManager import QuantumMachinesManager
from configuration import *
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
from qm import SimulationConfig
from qualang_tools.loops import from_array
%matplotlib qt

# Start to save data
import datetime 
import os
import pandas as pd 

now = datetime.datetime.now()
year = now.strftime("%Y")
month = now.strftime("%m")
day = now.strftime("%d")
hour = now.strftime("%H")
minute = now.strftime("%M")
second  = now.strftime("%S")

tPath = os.path.join(r'Z:\LabberData_DF5\QM_Data_DF5',year,month,'Data_'+month+day)
if not os.path.exists(tPath):
   os.makedirs(tPath)
exp_name = 'RR_spec_wAmplitude'
qubit_name = 'LA_Q2'

f_str =exp_name + '_'+ qubit_name +'_'+ hour+ '_' + minute +'_' + second
f_name= f_str+'.csv'
j_name = f_str + '_state.json'
p_name= f_str+'.png'

# End to save data

#########################################
# Set-up the machine and get the config #
#########################################
machine = QuAM("quam_state.json")
config = build_config(machine)

###################
# The QUA program #
###################

n_avg = 1000

cooldown_time = 20_000 * u.ns

amps = np.arange(0.1, 1.99, 0.10)
dfs = np.arange(-2.0e6, +2.0e6, 0.1e6)

qubit_index = 5
res_if = machine.resonators[qubit_index].f_readout - machine.resonators[qubit_index].lo


with program() as resonator_spec_2D:
    n = declare(int)  # Averaging index
    df = declare(int)  # Resonator frequency
    a = declare(fixed)  # Readout amplitude pre-factor
    I = declare(fixed)
    Q = declare(fixed)
    I_st = declare_stream()
    Q_st = declare_stream()
    n_st = declare_stream()

    with for_(n, 0, n < n_avg, n + 1):
        with for_(*from_array(df, dfs)):
            # Update the resonator frequency
            update_frequency(machine.resonators[qubit_index].name, df + res_if)
            with for_(*from_array(a, amps)):
                # Measure the resonator
                measure(
                    "readout" * amp(a),
                    machine.resonators[qubit_index].name,
                    None,
                    dual_demod.full("cos", "out1", "sin", "out2", I),
                    dual_demod.full("minus_sin", "out1", "cos", "out2", Q),
                )
                # Wait for the resonator to cooldown
                wait(cooldown_time, machine.resonators[qubit_index].name)
                # Save data to the stream processing
                save(I, I_st)
                save(Q, Q_st)
        save(n, n_st)

    with stream_processing():
        I_st.buffer(len(amps)).buffer(len(dfs)).average().save("I")
        Q_st.buffer(len(amps)).buffer(len(dfs)).average().save("Q")
        n_st.save("iteration")

#####################################
#  Open Communication with the QOP  #
#####################################
qmm = QuantumMachinesManager(machine.network.qop_ip, cluster_name=machine.network.cluster_name, octave=octave_config)

#######################
# Simulate or execute #
#######################

simulate = False

if simulate:
    simulation_config = SimulationConfig(duration=1000)
    job = qmm.simulate(config, resonator_spec_2D, simulation_config)
    job.get_simulated_samples().con1.plot()

else:
    qm = qmm.open_qm(config)
    job = qm.execute(resonator_spec_2D)
    # Get results from QUA program
    results = fetching_tool(job, data_list=["I", "Q", "iteration"], mode="live")
    # Live plotting
    fig = plt.figure()
    plt.rcParams['figure.figsize'] = [12, 8]
    interrupt_on_close(fig, job)  #  Interrupts the job when closing the figure
    
    while results.is_processing():
        # Fetch results
        I, Q, iteration = results.fetch_all()
        # Normalize data
        s1 = u.demod2volts(I + 1j * Q, machine.resonators[qubit_index].readout_pulse_length)
        A1 = np.abs(s1)
        row_sums = A1.sum(axis=0)
        A1 = A1 / row_sums[np.newaxis, :]
        
        # Progress bar
        progress_counter(iteration, n_avg, start_time=results.get_start_time())
        
        # 2D spectroscopy plot
        plt.subplot(211)
        plt.cla()
        plt.title("Resonator spectroscopy amplitude (normalized)")
        plt.pcolor(amps * machine.resonators[qubit_index].readout_pulse_amp, dfs / u.MHz, A1,cmap="seismic")
        plt.xlabel("Readout amplitude [V]")
        plt.ylabel("Frequency [MHz]")
        plt.colorbar()
        # plt.axhline(y=0,color='k')
        plt.subplot(212)
        plt.cla()
        plt.title("Resonator spectroscopy phase")
        plt.pcolor(amps * machine.resonators[qubit_index].readout_pulse_amp, dfs / u.MHz, signal.detrend(np.unwrap(np.angle(I + 1j * Q))),cmap="seismic")
        plt.xlabel("Readout amplitude [V]")
        plt.ylabel("Frequency [MHz]")
        plt.colorbar()
        plt.pause(0.1)
        plt.tight_layout()



2023-08-17 14:57:50,920 - qm - ERROR    - Encountered connection error from QOP
Traceback (most recent call last):
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\api\base_api.py", line 27, in wrapped
    return func(*args, **kwargs)
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\api\frontend_api.py", line 79, in get_version
    response = run_async(self._stub.get_version(Empty(), timeout=self._timeout))
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\qm\utils\async_utils.py", line 37, in run_async
    return create_future(coroutine).result()  # type: ignore[no-any-return]
  File "C:\Users\painter\.conda\envs\labber-qm-env\lib\concurrent\futures\_base.py", line 444, in result
    return self.__get_result()
  File "C:\Users\painter\.conda\envs\labber-qm-env\lib\concurrent\futures\_base.py", line 389, in __get_result
    raise self._exception
  File "C:\Users\painter\AppData\Roaming\Python\Python38\site-packages\

C:\Users\painter\AppData\Local\Temp\ipykernel_20180\4247986820.py:132: MatplotlibDeprecationWarning: Auto-removal of overlapping axes is deprecated since 3.6 and will be removed two minor releases later; explicitly call ax.remove() as needed.
  plt.subplot(211)
C:\Users\painter\AppData\Local\Temp\ipykernel_20180\4247986820.py:140: MatplotlibDeprecationWarning: Auto-removal of overlapping axes is deprecated since 3.6 and will be removed two minor releases later; explicitly call ax.remove() as needed.
  plt.subplot(212)


C:\Users\painter\AppData\Local\Temp\ipykernel_20180\4247986820.py:132: MatplotlibDeprecationWarning: Auto-removal of overlapping axes is deprecated since 3.6 and will be removed two minor releases later; explicitly call ax.remove() as needed.
  plt.subplot(211)
C:\Users\painter\AppData\Local\Temp\ipykernel_20180\4247986820.py:140: MatplotlibDeprecationWarning: Auto-removal of overlapping axes is deprecated since 3.6 and will be removed two minor releases later; explicitly call ax.remove() as needed.
  plt.subplot(212)


C:\Users\painter\AppData\Local\Temp\ipykernel_20180\4247986820.py:132: MatplotlibDeprecationWarning: Auto-removal of overlapping axes is deprecated since 3.6 and will be removed two minor releases later; explicitly call ax.remove() as needed.
  plt.subplot(211)
C:\Users\painter\AppData\Local\Temp\ipykernel_20180\4247986820.py:140: MatplotlibDeprecationWarning: Auto-removal of overlapping axes is deprecated since 3.6 and will be removed two minor releases later; explicitly call ax.remove() as needed.
  plt.subplot(212)


C:\Users\painter\AppData\Local\Temp\ipykernel_20180\4247986820.py:132: MatplotlibDeprecationWarning: Auto-removal of overlapping axes is deprecated since 3.6 and will be removed two minor releases later; explicitly call ax.remove() as needed.
  plt.subplot(211)
C:\Users\painter\AppData\Local\Temp\ipykernel_20180\4247986820.py:140: MatplotlibDeprecationWarning: Auto-removal of overlapping axes is deprecated since 3.6 and will be removed two minor releases later; explicitly call ax.remove() as needed.
  plt.subplot(212)


...

Progress: [##################################################] 100.0% (n=1000/1000) --> elapsed time: 61.02s


C:\Users\painter\AppData\Local\Temp\ipykernel_20180\4247986820.py:132: MatplotlibDeprecationWarning: Auto-removal of overlapping axes is deprecated since 3.6 and will be removed two minor releases later; explicitly call ax.remove() as needed.
  plt.subplot(211)
C:\Users\painter\AppData\Local\Temp\ipykernel_20180\4247986820.py:140: MatplotlibDeprecationWarning: Auto-removal of overlapping axes is deprecated since 3.6 and will be removed two minor releases later; explicitly call ax.remove() as needed.
  plt.subplot(212)


Progress: [##################################################] 100.0% (n=1000/1000) --> elapsed time: 61.36s


C:\Users\painter\AppData\Local\Temp\ipykernel_20180\4247986820.py:132: MatplotlibDeprecationWarning: Auto-removal of overlapping axes is deprecated since 3.6 and will be removed two minor releases later; explicitly call ax.remove() as needed.
  plt.subplot(211)
C:\Users\painter\AppData\Local\Temp\ipykernel_20180\4247986820.py:140: MatplotlibDeprecationWarning: Auto-removal of overlapping axes is deprecated since 3.6 and will be removed two minor releases later; explicitly call ax.remove() as needed.
  plt.subplot(212)


Execution stopped by user!


In [6]:
machine.resonators[qubit_index].readout_pulse_amp = 0.15

In [7]:
machine._save("quam_state.json", flat_data=False)

Execution stopped by user!
Execution stopped by user!


In [ ]:
plt.savefig(os.path.join(tPath, p_name))
amplitude = amps * machine.resonators[qubit_index].readout_pulse_amp
## convert everything to lists so they are easy to parse in Matlab if desired
if isinstance(amplitude,np.ndarray) == True:
   amplitude1=amplitude.tolist()
else:  
    pass
if isinstance(dfs,np.ndarray) == True:
   dfs1=dfs.tolist()
else:  
    pass
if isinstance(I,np.ndarray) == True:
   I1=I.tolist()
else:  
    pass
if isinstance(Q,np.ndarray) == True:
   Q1=Q.tolist()
else:  
    pass

data = [('Variable', 'Value'), ('Amp', amplitude1),('Freq', dfs1), ('I', I1), ('Q', Q1)]
df = pd.DataFrame(data,index=None, columns=None)
df.to_csv(os.path.join(tPath, f_name),header=False, index=False)
open(os.path.join(tPath, j_name), "w").write(open("quam_state.json").read())